In [20]:
import pickle
import sqlite3
import numpy as np
import os
import random

db_name = 'Source/dblp.db'
conn = sqlite3.connect(db_name) 

In [21]:
with open ('unique.pickle' , 'rb') as file:
    unique=pickle.load(file)
with open ('indices.pickle' , 'rb') as file:
    indices=pickle.load(file)
    
with open ('author_abbvs.pickle' , 'rb') as file:
    author_abbvs=pickle.load(file)
    
with open ('Source/index2auth.pickle' , 'rb') as file:
    authors=pickle.load(file)
    
    
    
with open('Source/x_train.pickle','rb') as file:
    x_train=pickle.load(file)
with open('Source/y_train.pickle','rb') as file:
    y_train=pickle.load(file)
with open('Source/x_test.pickle','rb') as file:
    x_test=pickle.load(file)
with open('Source/y_test.pickle','rb') as file:
    y_test=pickle.load(file)
with open('Source/x_validate.pickle','rb') as file:
    x_validate=pickle.load(file)
with open('Source/y_validate.pickle','rb') as file:
    y_validate=pickle.load(file)

In [22]:
# get the number of authors per variation:
n=[len(x) for x in indices]  #each element represents the number of authors shared by the respective name variante
s= sorted(range(len(n)), key=lambda k: n[k], reverse=True)    #indices of n sorted 

In [23]:

exst=True
i=0
while ((exst == True)  and (i<len(s))):
    model_version=s[i]  #the next untrained model of the name with highest number of authors 
    task='task_'+str(model_version)+'.pickle'
    exst=os.path.exists("Training/Tasks/"+task)
    i+=1

In [24]:
if i>len(s):
    print ('all done . . . ')  #all tasks for all author names have been created 
else: 
    print ('Continue . . .')

Continue . . .


In [25]:
# get author idx who have the same variante
authors_idex=indices[model_version]
all_authors=[authors[idx] for idx in authors_idex]

In [26]:
a=[]
for auth in all_authors:
    query='SELECT combination_id FROM reference WHERE target_author= "'+auth+'";'
    cursor = conn.cursor()
    cursor.execute(query)
    tmp2 = cursor.fetchall()
    a.extend(tmp2)
    cursor.close()
new_x=[b[0] for b in a]

In [27]:
#model_version=1519374

In [28]:
#get new dataset
vector_id=np.in1d(np.array(x_train), new_x)#.nonzero()[0]  #get indices in the vector 
vector_id=np.where(vector_id==True)[0]
final_x=np.array(x_train)[vector_id]  #comb ID
final_y=np.array(y_train)[vector_id]   #do not forget to change the order from 0 to n
#num_ref_15= int((15*len(final_x))/70)

In [29]:
#convert old IDS to new ones from 0 to number of authors
(old_authors_ids, counts) = np.unique(final_y, return_counts=True)
new_authors_ids=range(0,len(old_authors_ids))

In [30]:
#y_train2=[new_authors_ids[old_authors_ids.index(a)] for a in final_y]
#much faster
d=[[a,np.where(final_y==old_authors_ids[a])[0]] for a in range(len(old_authors_ids))]
y_train2=np.zeros(len(final_y))
for a in d:
    y_train2[a[1]]=new_authors_ids[a[0]]

In [31]:
#save new and old ids to change testdata 

with open('Training/Miscs/new_authors_ids_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(new_authors_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Training/Miscs/old_authors_ids_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(old_authors_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
old_authors_ids=list(old_authors_ids)

In [34]:
final_x, y_train2 = zip(*sorted(zip(final_x, y_train2)))
y_train2=[int(i) for i in y_train2]
final_x=np.array(final_x)

# save new training set 
with open('Training/Data/y_train_new_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(y_train2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Training/Data/x_train_new_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(final_x, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
len(np.unique(y_train2))

2601

In [36]:
# validation set
ind=np.intersect1d(old_authors_ids, y_validate) #get authors that are present in train set (basically old authors ids)
vector_id=np.in1d(np.array(y_validate), ind).nonzero()[0]  #get indices in the vector 
final_x_val=np.array(x_validate)[vector_id]  #comb ID
final_y_val=np.array(y_validate)[vector_id]   #do not forget to change the order from 0 to n


In [37]:
#num_ref_15
len(final_x_val)

25214

In [38]:
'''
# go for the simpliest solution 
try:
    vector_id=random.sample(range(len(final_x_val)), num_ref_15)
    final_x_val=np.array(final_x_val)[vector_id]  #comb ID
    final_y_val=np.array(final_y_val)[vector_id]   #do not forget to change the order from 0 to n
except: 
    pass
'''

'\n# go for the simpliest solution \ntry:\n    vector_id=random.sample(range(len(final_x_val)), num_ref_15)\n    final_x_val=np.array(final_x_val)[vector_id]  #comb ID\n    final_y_val=np.array(final_y_val)[vector_id]   #do not forget to change the order from 0 to n\nexcept: \n    pass\n'

In [39]:
#y_valid2=[new_authors_ids[old_authors_ids.index(a)] for a in final_y_val]

d=[[a,np.where(final_y_val==old_authors_ids[a])[0]] for a in range(len(old_authors_ids))]
y_valid2=np.zeros(len(final_y_val))
for a in d:
    y_valid2[a[1]]=new_authors_ids[a[0]]

In [40]:
final_x_val, y_valid2 = zip(*sorted(zip(final_x_val, y_valid2)))
y_valid2=[int(i) for i in y_valid2]
final_x_val=np.array(final_x_val)


# save new validation set 
with open('Training/Data/y_validate_new_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(y_valid2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Training/Data/x_validate_new_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(final_x_val, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [41]:
# test set
ind=np.intersect1d(old_authors_ids, y_test) #get authors that are present in train set (basically old authors ids)
vector_id=np.in1d(np.array(y_test), ind).nonzero()[0]  #get indices in the vector 
final_x_test=np.array(x_test)[vector_id]  #comb ID
final_y_test=np.array(y_test)[vector_id]   #do not forget to change the order from 0 to n

In [42]:
'''
# go for the simpliest solution 
try:
    vector_id=random.sample(range(len(final_x_test)), num_ref_15)
    final_x_test=np.array(final_x_test)[vector_id]  #comb ID
    final_y_test=np.array(final_y_test)[vector_id]   #do not forget to change the order from 0 to n
except:
    pass
'''

'\n# go for the simpliest solution \ntry:\n    vector_id=random.sample(range(len(final_x_test)), num_ref_15)\n    final_x_test=np.array(final_x_test)[vector_id]  #comb ID\n    final_y_test=np.array(final_y_test)[vector_id]   #do not forget to change the order from 0 to n\nexcept:\n    pass\n'

In [43]:
#y_test2=[new_authors_ids[old_authors_ids.index(a)] for a in final_y_test]
d=[[a,np.where(final_y_test==old_authors_ids[a])[0]] for a in range(len(old_authors_ids))]
y_test2=np.zeros(len(final_y_test))
for a in d:
    y_test2[a[1]]=new_authors_ids[a[0]]

In [44]:
final_x_test, y_test2 = zip(*sorted(zip(final_x_test, y_test2)))
y_test2=[int(i) for i in y_test2]
final_x_test=np.array(final_x_test)



# save new test set 
with open('Training/Data/y_test_new_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(y_test2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Training/Data/x_test_new_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(final_x_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:
if (conn):
    conn.close()

In [46]:
with open('Training/Tasks/task_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(model_version, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Reshuffle data

In [47]:

model_version=1520689
import numpy as np
import random
import pickle
import sqlite3

In [48]:



# Load train, test and validation daataset
#with tf.device('/gpu:0'):
with open('Training/Miscs/new_authors_ids_'+str(model_version)+'.pickle','rb') as file:
    new_authors_ids=pickle.load(file)
with open('Training/Miscs/old_authors_ids_'+str(model_version)+'.pickle','rb') as file:
    old_authors_ids=pickle.load(file)
with open('Training/Data/x_train_new_'+str(model_version)+'.pickle','rb') as file:
    x_train=pickle.load(file)
with open('Training/Data/y_train_new_'+str(model_version)+'.pickle','rb') as file:
    y_train=pickle.load(file)
with open('Training/Data/x_validate_new_'+str(model_version)+'.pickle','rb') as file:
    x_validate=pickle.load(file)
with open('Training/Data/y_validate_new_'+str(model_version)+'.pickle','rb') as file:
    y_validate=pickle.load(file)
    
with open('Training/Data/x_test_new_'+str(model_version)+'.pickle','rb') as file:
    x_test=pickle.load(file)
with open('Training/Data/y_test_new_'+str(model_version)+'.pickle','rb') as file:
    y_test=pickle.load(file)
with open('Source/auth2index.pickle','rb') as file:
    auth2index=pickle.load(file)

In [49]:
len(x_train)

123312

In [50]:
db_name = 'Source/dblp.db'
conn = sqlite3.connect(db_name) 

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d


zipped_lists = zip(x_train, y_train)
sorted_zipped_lists = sorted(zipped_lists)
x_train=[]
y_train=[]
sorted_list1 = [(x_train.append(aa), y_train.append(bb)) for aa, bb in sorted_zipped_lists]

conn.row_factory = dict_factory
query = "SELECT * FROM reference WHERE combination_id in "+str(tuple(x_train))
cursor = conn.cursor()
cursor.execute(query)
records_train = cursor.fetchall()
cursor.close()


In [51]:
len(x_validate)

25214

In [52]:
zipped_lists = zip(x_validate, y_validate)
sorted_zipped_lists = sorted(zipped_lists)
x_validate=[]
y_validate=[]
sorted_list1 = [(x_validate.append(aa), y_validate.append(bb)) for aa, bb in sorted_zipped_lists]



conn.row_factory = dict_factory
query = "SELECT * FROM reference WHERE combination_id in "+str(tuple(x_validate))
cursor = conn.cursor()
cursor.execute(query)
records_validate = cursor.fetchall()
cursor.close()


In [53]:
len(x_test)

26238

In [54]:

zipped_lists = zip(x_test, y_test)
sorted_zipped_lists = sorted(zipped_lists)
x_test=[]
y_test=[]
sorted_list1 = [(x_test.append(aa), y_test.append(bb)) for aa, bb in sorted_zipped_lists]


conn.row_factory = dict_factory
query = "SELECT * FROM reference WHERE combination_id in "+str(tuple(x_test))
cursor = conn.cursor()
cursor.execute(query)
records_test = cursor.fetchall()
cursor.close()
if (conn):
    conn.close()



In [55]:
ref_author=[]
ref_ref=[]
ref_year=[]
for record in records_train:
    ref_ref.append(tuple(record.values())[1])
    ref_year.append(int(tuple(record.values())[8]))
for record in records_validate:
    ref_ref.append(tuple(record.values())[1])
    ref_year.append(int(tuple(record.values())[8]))
for record in records_test:
    ref_ref.append(tuple(record.values())[1])
    ref_year.append(int(tuple(record.values())[8]))
X=np.concatenate((x_train,x_validate,x_test))
Y=np.concatenate((y_train,y_validate,y_test))
X=np.array(X)
Y=np.array(Y)

In [56]:
len(X)

174764

In [57]:

w=np.unique(Y)
ref_ref=np.array(ref_ref)
ref_year=np.array(ref_year)
n_x_train=[]
n_y_train=[]
n_x_validate=[]
n_y_validate=[]
n_x_test=[]
n_y_test=[]

tt=[]
tt2=[]
tt3=[]
for i in range(len(w)):
    tmp=np.where(Y==w[i])[0]
    tmp2=ref_ref[tmp]
    tmp3=np.unique(tmp2)
    
    '''
    tmp22=ref_year[tmp]
    tmp3,tmp30=np.unique(tmp2,return_index=True)
    tmp33=tmp22[tmp30]
    tmp0=np.argsort(tmp33)
    tmp3=tmp3[tmp0]
    tmp33=tmp33[tmp0]
    '''
    
    tp=int(0.7*len(tmp3))   # # traing samples
    tp=tp if tp>0 else 1
    vp=int(0.5*(len(tmp3)-tp))   # # validation samples
    random.shuffle(tmp3)
    tmp4=tmp3[:tp]
    tmp5=np.where(np.isin(ref_ref,tmp4))[0]
    tmp5=np.intersect1d(tmp5,tmp)
    if len(tmp5)<1:
        print ('stop')
        break
    #break
    n_x_train.extend(X[tmp5])
    n_y_train.extend(Y[tmp5])
    tt.extend(ref_ref[tmp5])
    
    tmp4=tmp3[tp:tp+vp]
    tmp5=np.where(np.isin(ref_ref,tmp4))[0]
    tmp5=np.intersect1d(tmp5,tmp)
    n_x_validate.extend(X[tmp5])
    n_y_validate.extend(Y[tmp5])
    tt2.extend(ref_ref[tmp5])
    
    tmp4=tmp3[tp+vp:]
    tmp5=np.where(np.isin(ref_ref,tmp4))[0]
    tmp5=np.intersect1d(tmp5,tmp)
    n_x_test.extend(X[tmp5])
    n_y_test.extend(Y[tmp5])
    tt3.extend(ref_ref[tmp5])

In [58]:
len(w)

2601

In [59]:
zipped_lists = zip(n_x_train, n_y_train)
sorted_zipped_lists = sorted(zipped_lists)
n_x_train=[]
n_y_train=[]
sorted_list1 = [(n_x_train.append(aa), n_y_train.append(bb)) for aa, bb in sorted_zipped_lists]


zipped_lists = zip(n_x_test, n_y_test)
sorted_zipped_lists = sorted(zipped_lists)
n_x_test=[]
n_y_test=[]
sorted_list1 = [(n_x_test.append(aa), n_y_test.append(bb)) for aa, bb in sorted_zipped_lists]

zipped_lists = zip(n_x_validate, n_y_validate)
sorted_zipped_lists = sorted(zipped_lists)
n_x_validate=[]
n_y_validate=[]
sorted_list1 = [(n_x_validate.append(aa), n_y_validate.append(bb)) for aa, bb in sorted_zipped_lists]

In [60]:
with open('Training/Data/y_test_new_reshuffeled_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(n_y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Training/Data/x_test_new_reshuffeled_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(n_x_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Training/Data/y_train_new_reshuffeled_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(n_y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Training/Data/x_train_new_reshuffeled_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(n_x_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Training/Data/y_validate_new_reshuffeled_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(n_y_validate, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Training/Data/x_validate_new_reshuffeled_'+str(model_version)+'.pickle', 'wb') as handle:
    pickle.dump(n_x_validate, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
v=-2
print (records_train[v])
print (x_train[v])

print (records_test[v])
print (x_test[v])

print (records_validate[v])
print (x_validate[v])

n_x_train

# END

# Some test that you can ignore

In [ ]:
#model_version=1520908
import pickle5 as pickle
import sqlite3

with open('Training/Data/y_test_new_reshuffeled_'+str(model_version)+'.pickle', 'rb') as handle:
    y_test=pickle.load(handle)
with open('Training/Data/x_test_new_reshuffeled_'+str(model_version)+'.pickle', 'rb') as handle:
    x_test=pickle.load(handle)
    
with open('Training/Data/y_train_new_reshuffeled_'+str(model_version)+'.pickle', 'rb') as handle:
    y_train=pickle.load(handle)
with open('Training/Data/x_train_new_reshuffeled_'+str(model_version)+'.pickle', 'rb') as handle:
    x_train=pickle.load(handle)
    
with open('Training/Data/y_validate_new_reshuffeled_'+str(model_version)+'.pickle', 'rb') as handle:
    y_validate=pickle.load(handle)
with open('Training/Data/x_validate_new_reshuffeled_'+str(model_version)+'.pickle', 'rb') as handle:
    x_validate=pickle.load(handle)

In [ ]:
print(len(X))
print(len(np.unique(X)))
print(len(n_x_test))
print(len(n_x_train))
print(len(n_x_validate))
n_x=np.concatenate((n_x_train,n_x_validate,n_x_test))
n_y=np.concatenate((n_y_train,n_y_validate,n_y_test))

'''
db_name = '../../../data/Bib2Auth/data1.db'
conn = sqlite3.connect(db_name) 

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d


zipped_lists = zip(n_x_train, n_y_train)
sorted_zipped_lists = sorted(zipped_lists)
n_x_train=[]
n_y_train=[]
sorted_list1 = [(n_x_train.append(aa), n_y_train.append(bb)) for aa, bb in sorted_zipped_lists]

conn.row_factory = dict_factory
query = "SELECT * FROM reference WHERE combination_id in "+str(tuple(n_x_train))
cursor = conn.cursor()
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
if (conn):
    conn.close()
for record in records:
    tt.append(tuple(record.values())[1])

'''
n_tt=np.concatenate((tt,tt2,tt3))
a,b=np.unique(n_tt,return_counts=True)
print(a)
print (b)
c=np.where(b>1)[0]
print(c)
d=a[c]
e=np.where(np.isin(n_x,d))[0]


In [ ]:
n_tt=np.concatenate((tt,tt2,tt3))
a,b=np.unique(n_tt,return_counts=True)


d0=0   #two authors

d1=0   #three authors
for aa in a:
    x=np.where(n_tt==aa)[0]
    x2=n_y[x]
    x3=np.unique(x2)
    if len(x3)>1:
        d0+=1
    if len(x3)>2:
        d1+=1   


In [ ]:
d1

In [ ]:
l0.
en(np.unique(n_x[e]))

In [ ]:
p=e[0]
print (n_tt[p])
print (n_x[p])

In [ ]:
n_tt=np.array(n_tt)
len(np.unique(n_tt[e]))

In [ ]:
ttt=np.unique(tt[e])
ttt[-1]
#np.where(tt==4825)
#n_x_train[[36951, 36952, 36953, 36954]]

In [ ]:
np.where(n_x_train==n_x_train[c[0]])

In [ ]:
d=n_x_train[e]
ee=np.where(np.isin(X,d))[0]
f=ref_ref[ee]
len(np.unique(f))
l=0
g=[]
for i in ee:
    record=records_train[i]
    if (tuple(record.values())[3]==tuple(record.values())[5])&(tuple(record.values())[4]==tuple(record.values())[6]):
        l+=1
        g.append(tuple(record.values())[1])

In [ ]:
a,b=np.unique(g,return_counts=True)
d=np.where(b>2)[0]

In [ ]:
a[d]

In [ ]:
a=np.where(ref_ref==5069)[0]
Y[a]

In [ ]:
records_train[a[-1]]

In [ ]:
tmp=np.where(Y==1566)[0]
tmp2=ref_ref[tmp]
tmp3=np.unique(tmp2)
print(tmp)

In [ ]:
a,b=np.unique(w,return_counts=True)
print (a)
print (b)
np.where(b>1)[0]

In [ ]:
a=[1,2,3]
b=[5,1,2,1,3,1,5,4]
np.where(np.isin(b,a))[0]